In [71]:
import torch
import torch_geometric
from torch_geometric.data import Data
import numpy as np
import networkx as nx
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import matplotlib.pyplot as plt
import scipy.special as SS
import scipy.stats as SSA
import torch.nn.functional as F

In [57]:
A = np.array([[0.25 , 0.25, 0.4, 0.1 ],
        [0.25, 0.75 , 0. , 0. ],
        [0.4, 0. , 0.55 , 0.05],
        [0.1 , 0 , 0.05, 0.85 ]])

In [59]:
g = nx.from_numpy_array(A)
data = torch_geometric.utils.from_networkx(g)
R0 = 2.5
# self.r = r ## to be estimated
r = torch.nn.Parameter(torch.tensor([0.1]))
p = r/(R0+r)
p_prime = 1 - p ## depends on R0

max_time_step = 5 ### probably i do not need it。
Z = 3 # latent period
Zb = 1 # scale parameter for Z
D = 5 # infectious period
Db = 1 # scale parameter for beta
# self.weight = torch.Tensor(weight)
offspring = []

x = torch.tensor(np.zeros((4,14)))## initials
x[1,0] = 8
x[2,1] = 5
x[3,5] = 10
population = pop = torch.tensor(np.array([10000]*4))
total_infection = torch.tensor(np.array([10,10,10,2])) ### using the real data

In [41]:
z_num = 1000000
latency_p = SSA.gamma.rvs(a=Z, scale=Zb, size=z_num)
infectious_p = SSA.gamma.rvs(a=D, scale=Db, size=z_num)
v = np.random.random(z_num)
delay_days = latency_p+v*infectious_p  # 3+5*0.5
### two weeks back to the past

In [130]:
back_time = 14
y = torch.tensor(np.zeros(4))
for step in range(back_time):
    new_infectors = x[:, step]
    temp = new_infectors.round().int()
    cases = temp.squeeze().tolist() ##case' indx is the node index

    results = torch.zeros_like(new_infectors)
    # print(new_infectors)
    source_nodes, target_nodes = data.edge_index
    messages = torch.zeros(data.num_nodes)
    # Generate negative binomial for each size
    for i, size in enumerate(cases): ###i is index for the edges
        if size > 0:
            offspring_per_case = torch.tensor([2]*size)
#             torch.distributions.negative_binomial.NegativeBinomial(r,p_prime).sample(sample_shape=torch.Size([size]))
            temp_sum = offspring_per_case.sum()
        else:
            temp_sum = 0
        messages[i] = temp_sum
    tt = torch.eye(4)
    m_messages = torch.diag_embed(messages) + tt - torch.diag(torch.diag(tt))
    # New code for sending messages based on edge weights the propograt can be here
#     for i, num_messages in enumerate(results):
#         for _ in range(int(num_messages)):
#             # For each message, choose a target node based on the edge probabilities
#             possible_targets = target_nodes[source_nodes == i]
#             target_probabilities = edge_attr[source_nodes == i]
#             chosen_target = torch.multinomial(target_probabilities, 1).item()
#             target_idx = (source_nodes == i) & (target_nodes == possible_targets[chosen_target])
#             messages[target_idx] += 1
    
    new_infections = m_messages@nx.adjacency_matrix(g).todense()
    population = pop
    rate = (population - total_infection) / population # Compute the rate. total_infection is an input
    rate[rate<0] = 0

    new_effective_infections = new_infections*rate
    new_infections_int  = new_effective_infections.round().int()
    ### diffuse the new_infections to different times
    inf_sizes = new_infections_int.squeeze().tolist()
    for i, inf_size_i in enumerate(inf_sizes):
        gamma_dist1 = torch.distributions.Gamma(Z, 1/Zb)
        gamma_dist2 = torch.distributions.Gamma(D, 1/Db)
        sizes = np.sum(inf_size_i)
        latency_p = gamma_dist1.rsample(sample_shape=torch.Size([sizes]))
        infectious_p = gamma_dist2.rsample(sample_shape=torch.Size([sizes]))
        v = torch.rand(sizes)
        delay_days = latency_p + v * infectious_p
#             print(step, delay_days)
        for j,delay_t in enumerate(delay_days):
            t_j = (step+delay_t).ceil().int()
            print(step, delay_t)
            if t_j == back_time: #### only record the infection at the time we are interested in
                y[i] = y[i] + 1

0 tensor(2.5230)
0 tensor(12.3609)
0 tensor(3.5490)
0 tensor(10.6438)
0 tensor(8.5996)
0 tensor(4.6597)
0 tensor(1.2012)
0 tensor(4.2412)
0 tensor(9.4676)
0 tensor(3.3354)
0 tensor(7.6250)
0 tensor(1.3635)
0 tensor(1.4792)
0 tensor(3.4435)
0 tensor(3.7236)
0 tensor(8.0827)
0 tensor(4.0163)
1 tensor(2.2201)
1 tensor(12.4158)
1 tensor(3.3944)
1 tensor(7.0561)
1 tensor(6.2439)
1 tensor(6.7322)
1 tensor(5.5401)
1 tensor(8.7803)
1 tensor(4.9264)
1 tensor(4.6999)
2 tensor(5.7611)
3 tensor(5.8678)
4 tensor(5.1355)
5 tensor(4.3139)
5 tensor(6.0825)
5 tensor(3.1200)
5 tensor(10.4837)
5 tensor(4.9924)
5 tensor(1.3521)
5 tensor(3.3610)
5 tensor(8.9523)
5 tensor(0.9444)
5 tensor(3.2114)
5 tensor(2.3849)
5 tensor(8.1534)
5 tensor(5.8184)
5 tensor(12.0369)
5 tensor(8.7426)
5 tensor(2.8049)
5 tensor(5.6131)
5 tensor(4.0394)
5 tensor(9.5898)
5 tensor(5.7604)
5 tensor(10.2747)
6 tensor(2.6483)
7 tensor(6.7478)
8 tensor(11.3939)
9 tensor(8.9851)
10 tensor(13.3437)
11 tensor(3.0602)
12 tensor(3.2915)
13 

In [131]:
y

tensor([0., 1., 1., 3.], dtype=torch.float64)